In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging

# import Pysyft to help us to simulate federated leraning
import syft as sy

# hook PyTorch to PySyft i.e. add extra functionalities to support Federated Learning
# and other private AI tools
hook = sy.TorchHook(torch) 

# we create two imaginary schools
jack = sy.VirtualWorker(hook, id="jack")
joe = sy.VirtualWorker(hook, id="joe")

In [2]:
# define the args
args = {
    'use_cuda' : False,
    'batch_size' : 8,
    'test_batch_size' : 8,
    'lr' : 0.01,
    'log_interval' : 10,
    'epochs' : 10
}

# check to use GPU or not
use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
# pytorch mlp for regression
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 32)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(32, 24)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        
        self.hidden3 = Linear(24, 16)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
        # third hidden layer and output
        self.hidden4 = Linear(16, 1)
        xavier_uniform_(self.hidden4.weight)

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        # third hidden layer and output
        X = self.hidden4(X)
        return X

In [4]:
path = './data/qoe/Client1_2.csv'
df = read_csv(path, header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,49450,54,3719,1108,0.0,3795,0,30013,30013,0,2934,1645944,4.33,3.96
1,50517,52,5902,1095,0.0,5859,0,30006,30006,0,2903,1555356,4.33,3.96
2,47988,47,5806,1051,0.0,5857,0,30006,30006,0,2903,1535444,4.33,3.98
3,56457,55,5978,1099,0.0,5989,0,30003,30003,0,2903,1534092,4.33,3.96
4,56690,54,5931,1133,0.0,5932,0,30005,30005,0,2934,1582192,4.33,3.95


In [5]:
path = './data/qoe/Client2_2.csv'
df2 = read_csv(path, header=None)
df2.head()

train_set_2=df2.sample(frac=0.8,random_state=200) #random state is a seed value
test_set_2=df2.drop(train_set_2.index)


In [6]:

train_set=df.sample(frac=0.8,random_state=200) #random state is a seed value
test_set=df.drop(train_set.index)




In [7]:
x  = train_set.iloc[: , :-1]
y = train_set.iloc[: , -1]

y = y.values.reshape((len(y), 1))
    

x_test  = test_set.iloc[: , :-1]
y_test = test_set.iloc[: , -1]

train_set_data = torch.from_numpy(x.to_numpy()).float()
target_set_data  = torch.from_numpy(y).float()

test_set_data = torch.from_numpy(x_test.to_numpy()).float()
target_test_set_data = torch.from_numpy(y_test.to_numpy()).float()



In [8]:
x2  = train_set_2.iloc[: , :-1]
y2 = train_set_2.iloc[: , -1]

y2 = y2.values.reshape((len(y2), 1))
    
x_test2  = test_set_2.iloc[: , :-1]
y_test2 = test_set_2.iloc[: , -1]

train_set_data2 = torch.from_numpy(x2.to_numpy()).float()
target_set_data2  = torch.from_numpy(y2).float()

test_set_data2 = torch.from_numpy(x_test2.to_numpy()).float()
target_test_set_data2 = torch.from_numpy(y_test2.to_numpy()).float()

In [9]:
import pandas as pd
import torch

# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [10]:
x.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12
674,21192,69,5416,1400,0.0,5545,0,30039,30039,0,2934,1554206,4.33
233,41173,63,5811,1115,0.0,6055,0,30005,30005,0,2903,1594692,4.33
739,29756,22,6199,1017,0.0,6380,0,30003,30003,0,2966,1597492,4.34
865,26345,38,6242,1275,0.0,6217,0,30003,30003,0,2966,1567349,4.34
523,20125,72,5910,1830,0.0,6054,0,30003,30003,0,2934,1556932,4.33


In [11]:
y.shape

(800, 1)

In [12]:
#x_df_tensor = df_to_tensor(train_set_data)
#y_df_tensor = df_to_tensor(target_set_data)
#test_tensor = df_to_tensor(test_set)


In [13]:
# Now we take the help of PySyft's awesome API to prepare the data for us and
# distribute for us across 2 workers ie. two schools
# normally we dont have to distribute data, data is already there at the site.
# We are doing this just to simulate federated learning.
# Below code looks just like torch code with just some minor changes. This is what's nice about PySyft.

target_set_data = target_set_data.type(torch.LongTensor)

bob_train_dataset = sy.BaseDataset(train_set_data,target_set_data).send(jack) 
anne_train_dataset = sy.BaseDataset(train_set_data2, target_set_data2).send(joe)



    
    
#series_tensor = df_to_tensor(series)
#federated_train_loader = sy.FederatedDataLoader(bob_train_dataset, ,batch_size=32, shuffle=False, drop_last=False)



#federated_train_loader = sy.FederatedDataLoader(
#    df_tensor.federate((jake, john)), batch_size=64, shuffle=True)

#test_loader = torch.utils.data.DataLoader(test_tensor, batch_size=32, shuffle=True)
federated_train_dataset= sy.FederatedDataset([bob_train_dataset, anne_train_dataset])
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=False, batch_size=8)

#federated_train_loader = sy.FederatedDataLoader(
#    datasets.MNIST('../data', train=True, download=True,
                 #  transform=transforms.Compose([
#                       transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
#                   ]))
#    .federate((grapevine_high, westside_school)),
#    batch_size=args['batch_size'], shuffle=True)




# test data remains with us locally
# this is the normal torch code to load test data from MNIST
# that we are all familiar with


In [14]:
print(federated_train_dataset)

FederatedDataset
    Distributed accross: jack, joe
    Number of datapoints: 1600



In [15]:
inputs, labels = next(iter(federated_train_loader))

In [16]:
print(labels)

(Wrapper)>[PointerTensor | me:96924076916 -> jack:49864387510]


In [17]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = MSELoss()
    # iterate over federated data
    for batch_idx, (data, target) in enumerate(train_loader):

        # send the model to the remote location 
        model = model.send(data.location)
        optimizer.zero_grad()
        # the same torch code that we are use to
        data, target = data.to(device), target.to(device)
        output = model(data)

        # this loss is a ptr to the tensor loss 
        # at the remote location
        #loss = F.nll_loss(output, target)
        #loss = F.mse_loss(output.view(-1), target.float())
        loss = criterion(output, target.float())

        # call backward() on the loss ptr,
        # that will send the command to call
        # backward on the actual loss tensor
        # present on the remote machine
        loss.backward()

        optimizer.step()

        # get back the updated model
        model.get()

        if batch_idx % args['log_interval'] == 0:

            # a thing to note is the variable loss was
            # also created at remote worker, so we need to
            # explicitly get it back
            loss = loss.get()

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader), 
                    loss.item()
                )
            )

In [18]:
def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    prediction = model(data)
    loss = F.mse_loss(prediction.view(-1), target)
    loss.backward()
    optimizer.step()
    return model

In [19]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()


In [20]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            print(data)
            print(target)
            output = model(data)
            # add losses together
            test_loss += F.mse_loss(output, target.float(), reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc4 = nn.Linear(24, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        #x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(x))
        x = self.fc3(x)
        return x

In [ ]:
#model = Net().to(device)
model = MLP(13).to(device)
#model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'])


logging.info("Starting training !!")

for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch)
        #test(model, device, test_loader)
    

In [22]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, :-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
    
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [23]:

# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl


In [24]:
path = './data/qoe/Client3.csv'
df3 = prepare_data(path)
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))

#test_loader =torch.utils.data.Dataset(test_dl)



938 462


In [ ]:
test(model, device, test_dl)

In [25]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [ ]:
#row2 = [34011,50,5699,1035,0,6015,0,30003,30003,0,2903,1544292,4.33]
row =  [57400,57,6023,1051,0,6128,0,30003,30003,0,2903,1565292,4.33] 
#= 3.98

client = predict(row, model)

In [ ]:
print('Predicted: %.3f' % client)

In [ ]:
# evaluate the model
def evaluate_model(test_dl, model):
    test_loss = 0
    correct = 0
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        output = model(inputs)
        test_loss += F.mse_loss(output, targets.float(), reduction='sum').item() 
        # retrieve numpy array
        output = output.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(output)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse


In [ ]:
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))